In [4]:
!pip install requests #installing request library

ERROR: Invalid requirement: '#installing'


In [5]:
# importing the library that we will use

import requests
import base64                       #to encode and decode our credentials
import datetime                     #to check credentials is expire 
from urllib.parse import urlencode  #application/x-www-form-urlencoded -> developer.spotify.com
import pandas as pd                 #to take output as .csv
import json                         #to read data because spotify use json format

In [6]:
client_id = 'b553881ee9534a33acc4550b3e6d13f0'
client_secret = '692d1e877dc64e3aa1f88218b0f4455f'

In [7]:
#this values are taken from spotify https://open.spotify.com/user/________

profile_aykol = "aykolm"
profile_can = "11101218134"
profile_buyukozer = "bykozerslyman"

In [8]:
#to avoid confusion, this class is created

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    #when creating object, the object will be created with our id and secret
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    #this method returns the client_creds
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("There is no client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    #Authorization: Basic <base64 encoded client_id:client_secret>    
    def get_token_headers(self):
        #defined get_client_credentials to get decoded client creds
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials" #grant_type -> Set it to client_credentials.
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers

###############################################################
###############################################################

    # used to get profile information : DisplayName, userId, FollowersCount, profileURL...
    def get_user_profile(self, _id = profile_aykol):
        headers = self.get_resource_header()
        profile_endpoint = f"https://api.spotify.com/v1/users"
        profile_data = _id.lower()
        profile_lookup_url = f"{profile_endpoint}/{profile_data}"
        r = requests.get(profile_lookup_url, headers = headers)
        if r.status_code not in range(200,299):
            return{}
        return r.json()
    
    # used to list playlists, we will use this method to access playlists' id
    def get_user_playlists(self, user_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/users/{user_id}/playlists"
        r = requests.get(endpoint, headers = headers,)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    #this method returns the playlist's contents
    def get_playlist(self, playlist_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}"
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    #returns playlist's all infos
    def get_playlists_items(self, playlist_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    #prints playlist's tracks
    #it is restricted by spotify to get maximum 100 tracks from playlist.
    def get_tracks_from_playlist(self, playlist_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        r = requests.get(endpoint, 
                         headers = headers,
                         params = {'fields' : 'items(track(name))'})
        deneme = r.json()
        row = 1
        for tracks in deneme['items']:
            print(row , '-\t',tracks['track']['name'])
            row = row+1
    
    #prints owner of the playlists, playlists name and playlists id
    def playlist_info(self, list_of_playlist):
        line = 1 
        for owner in list_of_playlist['items']:
            print ("Owner of the playlists: ", owner['owner']['display_name'], "\n")
            break
        for ids in list_of_playlist['items']:
            print(line, ''.ljust(1),'-'.ljust(1),ids['name'].ljust(40),ids['id'])     
            line = line +1 
    
    #returns tracks' ids
    def get_track_ids(self, playlist_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        r = requests.get(endpoint, headers = headers,
                        params = {'fields': 'items(track(name,id))'})
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    #returns track_meta[] -> will be used when creating csv file
    def get_tracks_info(self, track_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/tracks/{track_id}"
        r = requests.get(endpoint, headers = headers)
        meta = r.json()
        name = meta['name']
        artist = meta['album']['artists'][0]['name']
        duration = meta['duration_ms']
        duration /=1000
        popularity = meta['popularity']
        track_meta = [name,artist,round(duration),popularity]
        if r.status_code not in range(200,299):
            return {}
        return track_meta
 
    #returns features[] -> will be used when creating csv file
    def get_tracks_feature(self, track_id):
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/audio-features/{track_id}"
        r = requests.get(endpoint, headers = headers)
        features = r.json()
        danceability = features['danceability']
        energy = features['energy']
        loudness = features['loudness']
        speechiness = features['speechiness']
        tempo = features['tempo']
        valence = features['valence']
        features = [danceability, energy, loudness, speechiness, tempo, valence]
        if r.status_code not in range(200,299):
            return {}
        return features

In [9]:
#This will create an object with id and secret which are defined at start
spotify = SpotifyAPI(client_id, client_secret)

# Part: Reaching Data 

In [10]:
#returns all playlists' infos and save it list_of_playlist
list_of_playlist=spotify.get_user_playlists(profile_can)

#print the owner, playlists' name and ids
spotify.playlist_info(list_of_playlist)

Owner of the playlists:  Can Çiftçi 

1   - 🔥..                                      1eQqP5mSc3y73npxKnqYOA
2   - 💊                                        0R52ilHbM3lfmInXKcN8eA
3   - 🇷🇺                                       44Bi7tbNvDQcgcH43gNNzz
4   - 🌴                                        6j5ytzyc4JDIgGFD0z3WaV
5   - 🌊 🏄🏻‍♂️                                  6JDc6196pTc0SiWcWVpdeu
6   - C                                        23Xyu2w23rUtOSrmWNcJhv
7   - A                                        1asKF6vXPrsEhHNB2dWCze
8   - N                                        7Hq6Dqy0MXHIweipZ6xS7m
9   - 47                                       2WUeHDj4R9ZGhudNlgZLs8


In [11]:
tracks_with_id=spotify.get_track_ids("0R52ilHbM3lfmInXKcN8eA")

In [12]:
#to store data; for loop is defined which will provide track selected features song by song
track_meta_feature=[]
for track_meta in tracks_with_id['items']:
    #provide us track id
    id = track_meta['track']['id']
    #get info from selected id the add it to track_meta_feature
    track_meta_feature.append(spotify.get_tracks_info(id) + spotify.get_tracks_feature(id))

In [14]:
#to make understandable, columns are defined
df = pd.DataFrame(track_meta_feature,columns=['Name', 'Artist', 'Duration(s)', 'Popularity', 'Danceability', 'Energy', 
                                  'Loudness', 'Speechiness', 'Tempo', 'Valence'])
#to save .cvs file 
df.to_csv("can1", sep = ',')
df[0:10]

,Name,Artist,Duration(s),Popularity,Danceability,Energy,Loudness,Speechiness,Tempo,Valence
0,Before I Lose My Mind,Claptone,205,60,0.872,0.510,-8.433,0.0660,120.005,0.958
1,Force,Laolu,454,37,0.773,0.807,-9.562,0.0455,119.996,0.895
2,Paris,Else,210,69,0.792,0.583,-8.305,0.0270,102.979,0.540
3,Roads,Bristol,268,19,0.598,0.441,-6.973,0.0253,130.005,0.182
4,Transpiration,I:Cube,386,20,0.701,0.966,-5.552,0.0374,122.986,0.088
5,Hikâye,Santi & Tuğçe,466,38,0.650,0.573,-10.338,0.0384,97.017,0.215
6,Last Day - David August Revision,DAVID AUGUST,405,53,0.674,0.496,-14.709,0.0357,112.011,0.398
7,Birdsong,Claptone,167,43,0.643,0.549,-7.258,0.0295,118.961,0.262
8,Head & Heart (feat. MNEK),Joel Corry,166,93,0.734,0.874,-3.158,0.0662,122.953,0.905
9,Girl,KC Lights,159,53,0.628,0.917,-7.623,0.0428,122.976,0.261
